## Getting Started with Amazon Nova Models

The following examples will show how you can begin integrating with the Amazon Nova models.


## Model Invocation

#### Text Understanding

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

llm = ChatBedrock(
    model_id="us.amazon.nova-lite-v1:0",
    model_kwargs=dict(temperature=0.7)
)

messages = [
    ("system", "Provide three alternative song titles for a given user title"),
    ("user", "Teardrops on My Guitar"),
]

response = llm.invoke(messages)
print(f"Request ID: {response.id}")
response.pretty_print()


# Here we can pass the chat history to the model to ask follow up questions
multi_turn_messages = [
    *messages,
    response,
    HumanMessage(content="Select your favorite and tell me why"),
]

response = llm.invoke(multi_turn_messages)
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()

#### Streaming

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

llm = ChatBedrock(
    model_id="us.amazon.nova-lite-v1:0",
    model_kwargs=dict(temperature=0.7)
)

chain = llm | StrOutputParser()

messages = [
    SystemMessage(content="You are an author with experience writing creative novels"),
    HumanMessage(
        content="Write an outlin for a novel about a wizard named Theodore graduating from college"
    ),
]

for chunk in chain.stream(messages):
    print(chunk, end="")

## Agent Workflows

#### Binding Tools

We recommend when taking advantage of tool calling or agentic workflows to use greedy decoding values. For the Nova models, this means temperature=1, topP=1, topK=1

In [ ]:
from langchain_aws import ChatBedrock
from langchain.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools = [multiply]

llm_with_tools = ChatBedrock(
    model_id="us.amazon.nova-lite-v1:0",
    model_kwargs=dict(temperature=1, top_p=1, additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    })
).bind_tools(tools)

response = llm_with_tools.invoke([("user", "What is 8*8")])

print("[Model Response]\n")
print(response.content)

print("\n[Tool Calls]\n")
print(response.tool_calls)

#### Tool Calling Agents


In [ ]:
from langchain.agents import tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools = [multiply]

llm_with_tools = ChatBedrock(
    model_id="us.amazon.nova-lite-v1:0",
    model_kwargs=dict(temperature=1, top_p=1, additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    })
).bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is 2*2?"})

## Structured Output

Structured output is a great way to force the model to return in a specific way. We use Greedy Decoding params here for more determinist results (Temperature = 1, Top P = 1, Top K = 1)

In [ ]:
from pydantic import BaseModel, Field
from langchain_aws import ChatBedrock

class Joke(BaseModel):
    """A joke to respond to the user"""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")

llm = ChatBedrock(
    model="us.amazon.nova-lite-v1:0",
    model_kwargs=dict(temperature=1, top_p=1, additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    })
)

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")